In [13]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

if os.path.join('..', '..', 'recommenders') not in sys.path:
    sys.path.append(os.path.join('..', '..', 'recommenders'))

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


# Locally import the model
from models.deeprec.models.sequential.din import DIN_RECModel as SeqModel


#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

import pickle as pkl
from recommenders.models.deeprec.deeprec_utils import load_dict

System version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Tensorflow version: 2.9.0


In [10]:
sys.path

['/home/caozheng/Desktop/code/recommenders/examples/00_quick_start',
 '/home/caozheng/anaconda3/lib/python39.zip',
 '/home/caozheng/anaconda3/lib/python3.9',
 '/home/caozheng/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/recommenders-1.1.0-py3.9.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/scikit_surprise-1.1.1-py3.9-linux-x86_64.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/pandera-0.11.0-py3.9.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/retrying-1.3.3-py3.9.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/cornac-1.14.2-py3.9-linux-x86_64.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/PyYAML-5.4.1-py3.9-linux-x86_64.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/category_encoders-1.3.0-py3.9.egg',
 '/home/caozheng/anaconda3/lib/python3.9/site-packages/transformers-4.19.2-py3.9.egg',
 '/home/

In [26]:
yaml_file = '../../recommenders/models/deeprec/config/din.yaml'  

In [27]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join("..", "..", "tests", "resources", "deeprec", "slirec")

In [28]:

# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
# sample_rate = 0.01 # sample a small item set for training and testing here for fast example
sample_rate = 1

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

# data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

# To analyze the dataset of Amazon

number of users: 123,960

number of movies: 50,024

number of categories: 164

number of historical actions for each user:

number of train samples: 1,325,653

number of test samples: 1,239,600

number of pos samples: 

number of neg samples:

In [29]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
            )

In [30]:
hparams.values()

{'use_entity': True,
 'use_context': True,
 'cross_activation': 'identity',
 'user_dropout': True,
 'dropout': [0.3, 0.3],
 'attention_dropout': 0.0,
 'load_saved_model': False,
 'fast_CIN_d': 0,
 'use_Linear_part': False,
 'use_FM_part': False,
 'use_CIN_part': False,
 'use_DNN_part': False,
 'init_method': 'tnormal',
 'init_value': 0.01,
 'embed_l2': 0.0,
 'embed_l1': 0.0,
 'layer_l2': 0.0,
 'layer_l1': 0.0,
 'cross_l2': 0.0,
 'cross_l1': 0.0,
 'reg_kg': 0.0,
 'learning_rate': 0.001,
 'lr_rs': 1,
 'lr_kg': 0.5,
 'kg_training_interval': 5,
 'max_grad_norm': 2,
 'is_clip_norm': 0,
 'dtype': 32,
 'optimizer': 'adam',
 'epochs': 10,
 'batch_size': 400,
 'enable_BN': True,
 'show_step': 20,
 'save_model': True,
 'save_epoch': 1,
 'write_tfevents': True,
 'train_num_ngs': 4,
 'need_sample': True,
 'embedding_dropout': 0.0,
 'EARLY_STOP': 10,
 'min_seq_length': 1,
 'slots': 5,
 'cell': 'SUM',
 'user_vocab': '../../tests/resources/deeprec/slirec/user_vocab.pkl',
 'item_vocab': '../../tests/r

In [23]:
user_yaml = load_dict(hparams.user_vocab)
item_yaml = load_dict(hparams.item_vocab)
cate_yaml = load_dict(hparams.cate_vocab)

In [25]:
len(user_yaml), len(item_yaml), len(cate_yaml)

(123960, 50024, 164)

In [31]:
input_creator = SequentialIterator
#### uncomment this for the NextItNet model, because it needs a special data iterator for training
#input_creator = NextItNetIterator

In [11]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

2022-05-24 01:36:35.003906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 01:36:35.022400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 01:36:35.022595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/home/caozheng/anaconda3/lib/python3.9/site-packages/recommenders-1.1.0-py3.9.egg/recommenders/models/deeprec/models/base_model.py:701: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.Grap

step 20 , total_loss: 1.5962, data_loss: 1.5962
step 40 , total_loss: 1.5645, data_loss: 1.5645
step 60 , total_loss: 1.5578, data_loss: 1.5578
step 80 , total_loss: 1.5354, data_loss: 1.5354
step 100 , total_loss: 1.5639, data_loss: 1.5639
step 120 , total_loss: 1.5204, data_loss: 1.5204
step 140 , total_loss: 1.5545, data_loss: 1.5545
step 160 , total_loss: 1.5359, data_loss: 1.5359
step 180 , total_loss: 1.5305, data_loss: 1.5305
step 200 , total_loss: 1.5194, data_loss: 1.5194
step 220 , total_loss: 1.4985, data_loss: 1.4985
step 240 , total_loss: 1.5533, data_loss: 1.5533
step 260 , total_loss: 1.4546, data_loss: 1.4546
step 280 , total_loss: 1.4668, data_loss: 1.4668
step 300 , total_loss: 1.5295, data_loss: 1.5295
step 320 , total_loss: 1.4794, data_loss: 1.4794
step 340 , total_loss: 1.4892, data_loss: 1.4892
step 360 , total_loss: 1.4649, data_loss: 1.4649
step 380 , total_loss: 1.4442, data_loss: 1.4442
step 400 , total_loss: 1.4854, data_loss: 1.4854
step 420 , total_loss: 1

eval valid at epoch 1: auc:0.8234,logloss:0.5657,mean_mrr:0.7524,ndcg@2:0.7213,ndcg@4:0.8008,ndcg@6:0.8146,group_auc:0.814
step 20 , total_loss: 0.9565, data_loss: 0.9565
step 40 , total_loss: 0.8961, data_loss: 0.8961
step 60 , total_loss: 0.9166, data_loss: 0.9166
step 80 , total_loss: 1.0226, data_loss: 1.0226
step 100 , total_loss: 0.8890, data_loss: 0.8890
step 120 , total_loss: 0.9297, data_loss: 0.9297
step 140 , total_loss: 0.9306, data_loss: 0.9306
step 160 , total_loss: 0.8765, data_loss: 0.8765
step 180 , total_loss: 0.9649, data_loss: 0.9649
step 200 , total_loss: 0.9458, data_loss: 0.9458
step 220 , total_loss: 0.9578, data_loss: 0.9578
step 240 , total_loss: 0.9478, data_loss: 0.9478
step 260 , total_loss: 1.0113, data_loss: 1.0113
step 280 , total_loss: 0.8936, data_loss: 0.8936
step 300 , total_loss: 0.9532, data_loss: 0.9532
step 320 , total_loss: 0.9423, data_loss: 0.9423
step 340 , total_loss: 0.9704, data_loss: 0.9704
step 360 , total_loss: 0.8271, data_loss: 0.8271

step 3280 , total_loss: 0.9087, data_loss: 0.9087
step 3300 , total_loss: 0.8889, data_loss: 0.8889
eval valid at epoch 2: auc:0.8514,logloss:0.5886,mean_mrr:0.7838,ndcg@2:0.7593,ndcg@4:0.8279,ndcg@6:0.8383,group_auc:0.841
step 20 , total_loss: 0.7759, data_loss: 0.7759
step 40 , total_loss: 0.8570, data_loss: 0.8570
step 60 , total_loss: 0.8029, data_loss: 0.8029
step 80 , total_loss: 0.7823, data_loss: 0.7823
step 100 , total_loss: 0.8487, data_loss: 0.8487
step 120 , total_loss: 0.7812, data_loss: 0.7812
step 140 , total_loss: 0.8420, data_loss: 0.8420
step 160 , total_loss: 0.7717, data_loss: 0.7717
step 180 , total_loss: 0.8426, data_loss: 0.8426
step 200 , total_loss: 0.8598, data_loss: 0.8598
step 220 , total_loss: 0.8086, data_loss: 0.8086
step 240 , total_loss: 0.8192, data_loss: 0.8192
step 260 , total_loss: 0.9218, data_loss: 0.9218
step 280 , total_loss: 0.8046, data_loss: 0.8046
step 300 , total_loss: 0.8346, data_loss: 0.8346
step 320 , total_loss: 0.7909, data_loss: 0.79

step 3240 , total_loss: 0.8982, data_loss: 0.8982
step 3260 , total_loss: 0.7971, data_loss: 0.7971
step 3280 , total_loss: 0.8324, data_loss: 0.8324
step 3300 , total_loss: 0.8017, data_loss: 0.8017
eval valid at epoch 3: auc:0.8597,logloss:0.6299,mean_mrr:0.7973,ndcg@2:0.7753,ndcg@4:0.8389,ndcg@6:0.8484,group_auc:0.852
step 20 , total_loss: 0.7803, data_loss: 0.7803
step 40 , total_loss: 0.7818, data_loss: 0.7818
step 60 , total_loss: 0.6993, data_loss: 0.6993
step 80 , total_loss: 0.7842, data_loss: 0.7842
step 100 , total_loss: 0.7506, data_loss: 0.7506
step 120 , total_loss: 0.7033, data_loss: 0.7033
step 140 , total_loss: 0.7871, data_loss: 0.7871
step 160 , total_loss: 0.7303, data_loss: 0.7303
step 180 , total_loss: 0.7689, data_loss: 0.7689
step 200 , total_loss: 0.7822, data_loss: 0.7822
step 220 , total_loss: 0.8179, data_loss: 0.8179
step 240 , total_loss: 0.7720, data_loss: 0.7720
step 260 , total_loss: 0.7611, data_loss: 0.7611
step 280 , total_loss: 0.8572, data_loss: 0.

...

step 3120 , total_loss: 0.6497, data_loss: 0.6497
step 3140 , total_loss: 0.7040, data_loss: 0.7040
step 3160 , total_loss: 0.7166, data_loss: 0.7166
step 3180 , total_loss: 0.7923, data_loss: 0.7923
step 3200 , total_loss: 0.6941, data_loss: 0.6941
step 3220 , total_loss: 0.7387, data_loss: 0.7387
step 3240 , total_loss: 0.6739, data_loss: 0.6739
step 3260 , total_loss: 0.6853, data_loss: 0.6853
step 3280 , total_loss: 0.6998, data_loss: 0.6998
step 3300 , total_loss: 0.7076, data_loss: 0.7076
eval valid at epoch 6: auc:0.8742,logloss:0.7442,mean_mrr:0.8158,ndcg@2:0.7979,ndcg@4:0.8543,ndcg@6:0.8623,group_auc:0.8671
step 20 , total_loss: 0.6962, data_loss: 0.6962
step 40 , total_loss: 0.5327, data_loss: 0.5327
step 60 , total_loss: 0.6270, data_loss: 0.6270
step 80 , total_loss: 0.6917, data_loss: 0.6917
step 100 , total_loss: 0.6974, data_loss: 0.6974
step 120 , total_loss: 0.6651, data_loss: 0.6651
step 140 , total_loss: 0.6398, data_loss: 0.6398
step 160 , total_loss: 0.6102, data_l

step 3080 , total_loss: 0.6374, data_loss: 0.6374
step 3100 , total_loss: 0.6679, data_loss: 0.6679
step 3120 , total_loss: 0.7028, data_loss: 0.7028
step 3140 , total_loss: 0.6626, data_loss: 0.6626
step 3160 , total_loss: 0.6484, data_loss: 0.6484
step 3180 , total_loss: 0.6921, data_loss: 0.6921
step 3200 , total_loss: 0.7183, data_loss: 0.7183
step 3220 , total_loss: 0.6853, data_loss: 0.6853
step 3240 , total_loss: 0.6661, data_loss: 0.6661
step 3260 , total_loss: 0.6780, data_loss: 0.6780
step 3280 , total_loss: 0.7619, data_loss: 0.7619
step 3300 , total_loss: 0.6539, data_loss: 0.6539
eval valid at epoch 7: auc:0.876,logloss:0.7891,mean_mrr:0.8182,ndcg@2:0.8008,ndcg@4:0.8563,ndcg@6:0.8641,group_auc:0.8691
step 20 , total_loss: 0.6765, data_loss: 0.6765
step 40 , total_loss: 0.5686, data_loss: 0.5686
step 60 , total_loss: 0.6469, data_loss: 0.6469
step 80 , total_loss: 0.6122, data_loss: 0.6122
step 100 , total_loss: 0.6012, data_loss: 0.6012
step 120 , total_loss: 0.6630, data_

step 3040 , total_loss: 0.6103, data_loss: 0.6103
step 3060 , total_loss: 0.6177, data_loss: 0.6177
step 3080 , total_loss: 0.6606, data_loss: 0.6606
step 3100 , total_loss: 0.6705, data_loss: 0.6705
step 3120 , total_loss: 0.6434, data_loss: 0.6434
step 3140 , total_loss: 0.7198, data_loss: 0.7198
step 3160 , total_loss: 0.6717, data_loss: 0.6717
step 3180 , total_loss: 0.6523, data_loss: 0.6523
step 3200 , total_loss: 0.6631, data_loss: 0.6631
step 3220 , total_loss: 0.6549, data_loss: 0.6549
step 3240 , total_loss: 0.6781, data_loss: 0.6781
step 3260 , total_loss: 0.6760, data_loss: 0.6760
step 3280 , total_loss: 0.7392, data_loss: 0.7392
step 3300 , total_loss: 0.6817, data_loss: 0.6817
eval valid at epoch 8: auc:0.8765,logloss:0.8245,mean_mrr:0.8197,ndcg@2:0.8027,ndcg@4:0.8574,ndcg@6:0.8652,group_auc:0.8702
step 20 , total_loss: 0.5704, data_loss: 0.5704
step 40 , total_loss: 0.5186, data_loss: 0.5186
step 60 , total_loss: 0.5409, data_loss: 0.5409
step 80 , total_loss: 0.5708, da

step 3000 , total_loss: 0.6278, data_loss: 0.6278
step 3020 , total_loss: 0.5987, data_loss: 0.5987
step 3040 , total_loss: 0.6115, data_loss: 0.6115
step 3060 , total_loss: 0.6976, data_loss: 0.6976
step 3080 , total_loss: 0.6739, data_loss: 0.6739
step 3100 , total_loss: 0.6482, data_loss: 0.6482
step 3120 , total_loss: 0.5961, data_loss: 0.5961
step 3140 , total_loss: 0.5792, data_loss: 0.5792
step 3160 , total_loss: 0.6207, data_loss: 0.6207
step 3180 , total_loss: 0.6952, data_loss: 0.6952
step 3200 , total_loss: 0.6202, data_loss: 0.6202
step 3220 , total_loss: 0.6633, data_loss: 0.6633
step 3240 , total_loss: 0.6344, data_loss: 0.6344
step 3260 , total_loss: 0.6568, data_loss: 0.6568
step 3280 , total_loss: 0.6090, data_loss: 0.6090
step 3300 , total_loss: 0.5851, data_loss: 0.5851
eval valid at epoch 9: auc:0.8779,logloss:0.8184,mean_mrr:0.8211,ndcg@2:0.8036,ndcg@4:0.8585,ndcg@6:0.8663,group_auc:0.8709
step 20 , total_loss: 0.5905, data_loss: 0.5905
step 40 , total_loss: 0.5659

step 2960 , total_loss: 0.5635, data_loss: 0.5635
step 2980 , total_loss: 0.6342, data_loss: 0.6342
step 3000 , total_loss: 0.6068, data_loss: 0.6068
step 3020 , total_loss: 0.6091, data_loss: 0.6091
step 3040 , total_loss: 0.6741, data_loss: 0.6741
step 3060 , total_loss: 0.6041, data_loss: 0.6041
step 3080 , total_loss: 0.6068, data_loss: 0.6068
step 3100 , total_loss: 0.5978, data_loss: 0.5978
step 3120 , total_loss: 0.6106, data_loss: 0.6106
step 3140 , total_loss: 0.6288, data_loss: 0.6288
step 3160 , total_loss: 0.6017, data_loss: 0.6017
step 3180 , total_loss: 0.6462, data_loss: 0.6462
step 3200 , total_loss: 0.5918, data_loss: 0.5918
step 3220 , total_loss: 0.6137, data_loss: 0.6137
step 3240 , total_loss: 0.6285, data_loss: 0.6285
step 3260 , total_loss: 0.6276, data_loss: 0.6276
step 3280 , total_loss: 0.6439, data_loss: 0.6439
step 3300 , total_loss: 0.6080, data_loss: 0.6080
eval valid at epoch 10: auc:0.8761,logloss:0.8605,mean_mrr:0.8197,ndcg@2:0.8021,ndcg@4:0.8572,ndcg@6

In [1]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)

NameError: name 'model' is not defined

In [11]:
model_best_trained = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
path_best_trained = os.path.join(hparams.MODEL_DIR, "best_model")
print('loading saved model in {0}'.format(path_best_trained))
model_best_trained.load_model(path_best_trained)

2022-05-24 08:41:37.748275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 08:41:37.799293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 08:41:37.799487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/home/caozheng/anaconda3/lib/python3.9/site-packages/recommenders-1.1.0-py3.9.egg/recommenders/models/deeprec/models/base_model.py:701: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.Grap

loading saved model in ../../tests/resources/deeprec/slirec/model/best_model


In [12]:
model_best_trained.run_eval(test_file, num_ngs=test_num_ngs)

2022-05-24 08:42:20.828762: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


{'auc': 0.8729,
 'logloss': 0.8969,
 'mean_mrr': 0.7128,
 'ndcg@2': 0.6716,
 'ndcg@4': 0.7388,
 'ndcg@6': 0.7642,
 'group_auc': 0.8648}